In [1]:
push!(LOAD_PATH, normpath(@__DIR__, "../../", "src/models"));
push!(LOAD_PATH, normpath(@__DIR__, "../../", "src/processing"));
ENV["COLUMNS"] = 200;

In [2]:
using Dates
using JuMP
using CSV
using DataFrames
using LinearAlgebra

In [3]:
using BedsData
using ForecastData
using GeographicData

In [4]:
using ReusableResourceAllocation

In [5]:
ne_states = ["CT", "DE", "MA", "MD", "ME", "NH", "NJ", "NY", "PA", "RI", "VT"]
start_date = Date(2020, 4, 1)
end_date   = Date(2020, 5, 1)
pct_beds_available = 0.25
travel_threshold_hours = 4.0;

In [6]:
beds = compute_beds(ne_states, pct_beds_available=pct_beds_available);
forecast_start, forecast_net = compute_ihme_forecast_net(start_date, end_date, ne_states);
adj = compute_adjacencies(ne_states, hrs=travel_threshold_hours);

In [7]:
model = patient_allocation(
    beds,
    forecast_start,
    forecast_net,
    adj,
    send_new_only=true,
    send_wait_period=5,
    min_send_amt=10,
    smoothness_penalty=0.1,
    setup_cost=0.1,
    sent_penalty=0,
    verbose=true
)
sent = value.(model[:sent])
println("termination status: ", termination_status(model))
println("solve time: ", round(solve_time(model), digits=3), "s")
println("objective function value: ", round(objective_value(model), digits=3))

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 8765 rows, 8547 columns and 162001 nonzeros
Model fingerprint: 0x009afa57
Model has 385 SOS constraints
Variable types: 4741 continuous, 55 integer (55 binary)
Semi-Variable types: 3751 continuous, 0 integer
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-01, 1e+00]
  Bounds range     [1e+01, 1e+01]
  RHS range        [3e-02, 1e+04]
Presolve removed 6330 rows and 6125 columns
Presolve time: 0.21s
Presolved: 4285 rows, 3347 columns, 48517 nonzeros
Presolved model has 119 SOS constraint(s)
Variable types: 2296 continuous, 1051 integer (1051 binary)
Found heuristic solution: objective 107644.47627

Root relaxation: objective 2.101548e+02, 3160 iterations, 0.17 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent   

In [8]:
overflow = i -> sum(max.(0,
    forecast_start[i]
    + sum(forecast_net[i,1:t])
    - sum(sent[i,:,1:t-1])
    + sum(sent[:,i,1:t])
    - beds[i])
    for t=1:size(sent,3)
);

In [9]:
println("Total overflow: ", sum(overflow.(1:length(ne_states))))

Total overflow: 0.0004920959522678459


In [10]:
summary = DataFrame(
    state=ne_states,
    total_sent=sum(sent, dims=[2,3])[:],
    total_received=sum(sent, dims=[1,3])[:],
    overflow=overflow.(1:length(ne_states)),
)

,state,total_sent,total_received,overflow
,String,Float64,Float64,Float64
1,CT,3570.35,0.0,0.000488281
2,DE,0.0,479.145,0.0
3,MA,1429.12,0.0,0.0
4,MD,0.0,135.691,0.0
5,ME,0.0,805.962,0.0
6,NH,0.0,687.997,3.8147e-6
7,NJ,1683.44,741.126,9.09495e-13
8,NY,3537.92,2251.17,0.0
9,PA,0.0,5085.67,0.0


In [11]:
sent_matrix = DataFrame(sum(sent, dims=3)[:,:,1])
rename!(sent_matrix, Symbol.(ne_states))
insertcols!(sent_matrix, 1, :state => ne_states)

,state,CT,DE,MA,MD,ME,NH,NJ,NY,PA,RI,VT
,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,CT,0.0,479.145,0.0,0.0,805.962,0.0,0.0,2041.17,0.0,0.0,244.071
2,DE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,MA,0.0,0.0,0.0,0.0,0.0,687.997,741.126,0.0,0.0,0.0,0.0
4,MD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,ME,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,NH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,NJ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1683.44,0.0,0.0
8,NY,0.0,0.0,0.0,135.691,0.0,0.0,0.0,0.0,3402.23,0.0,0.0
9,PA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
sent_vis_matrix = sum(sent, dims=3)[:,:,1] + diagm(sum(max.(0, forecast_net), dims=2)[:] - sum(sent, dims=[2,3])[:])
sent_vis_matrix = DataFrame(sent_vis_matrix)
rename!(sent_vis_matrix, Symbol.(ne_states));
# CSV.write("patient_sent_matrix.csv", sent_vis_matrix);

In [13]:
n_days = size(sent,3)
outcomes = DataFrame()
for (i,s) in enumerate(ne_states)
    single_state_outcome = DataFrame(
        state=fill(s, n_days),
        day=start_date .+ Dates.Day.(0:n_days-1),
        sent=sum(sent[i,:,:], dims=1)[:],
        received=sum(sent[:,i,:], dims=1)[:],
        net_patients=forecast_net[i,:],
        total_patients=[forecast_start[i] + sum(forecast_net[i,1:t]) - sum(sent[i,:,1:t]) + sum(sent[:,i,1:t]) for t in 1:n_days],
        capacity=fill(beds[i], n_days),
        overflow=[max(0, forecast_start[i] + sum(forecast_net[i,1:t]) - sum(sent[i,:,1:t-1]) + sum(sent[:,i,1:t]) - beds[i]) for t in 1:n_days],
        sent_to=[sum(sent[i,:,t])>0 ? collect(zip(ne_states[sent[i,:,t] .> 0], sent[i,sent[i,:,t].>0,t])) : "[]" for t in 1:n_days],
        sent_from=[sum(sent[:,i,t])>0 ? collect(zip(ne_states[sent[:,i,t] .> 0], sent[sent[:,i,t].>0,i,t])) : "[]" for t in 1:n_days],
    )
    outcomes = vcat(outcomes, single_state_outcome)
end
# CSV.write("patient_allocation_results.csv", outcomes)
println("First day:")
filter(row -> row.day == start_date, outcomes)

First day:


,state,day,sent,received,net_patients,total_patients,capacity,overflow,sent_to,sent_from
,String,Date,Float64,Float64,Float32,Float64,Float32,Float64,Any,Any
1,CT,2020-04-01,41.2727,0.0,41.2727,744.451,2159.75,0.0,"[(""DE"", 10.0), (""ME"", 31.2727)]",[]
2,DE,2020-04-01,0.0,10.0,1.51812,57.4492,572.5,0.0,[],"[(""CT"", 10.0)]"
3,MA,2020-04-01,27.7299,0.0,39.803,569.817,3790.75,0.0,"[(""NH"", 27.7299)]",[]
4,MD,2020-04-01,0.0,19.3844,147.355,594.846,2793.5,0.0,[],"[(""NY"", 19.3844)]"
5,ME,2020-04-01,0.0,31.2727,-2.0,66.8066,887.5,0.0,[],"[(""CT"", 31.2727)]"
6,NH,2020-04-01,0.0,27.7299,-3.65505,57.8255,755.0,0.0,[],"[(""MA"", 27.7299)]"
7,NJ,2020-04-01,213.601,0.0,213.601,3667.08,6239.5,0.0,"[(""PA"", 213.601)]",[]
8,NY,2020-04-01,505.417,0.0,832.831,10020.3,13039.5,0.0,"[(""MD"", 19.3844), (""PA"", 486.033)]",[]
9,PA,2020-04-01,0.0,699.634,130.308,1591.73,9372.25,0.0,[],"[(""NJ"", 213.601), (""NY"", 486.033)]"


In [14]:
s = "NJ"
filter(row -> row.state == s, outcomes)

,state,day,sent,received,net_patients,total_patients,capacity,overflow,sent_to,sent_from
,String,Date,Float64,Float64,Float32,Float64,Float32,Float64,Any,Any
1,NJ,2020-04-01,213.601,0.0,213.601,3667.08,6239.5,0.0,"[(""PA"", 213.601)]",[]
2,NJ,2020-04-02,226.853,0.0,774.71,4214.94,6239.5,0.0,"[(""PA"", 226.853)]",[]
3,NJ,2020-04-03,226.853,0.0,819.75,4807.83,6239.5,0.0,"[(""PA"", 226.853)]",[]
4,NJ,2020-04-04,226.853,0.0,228.243,4809.22,6239.5,0.0,"[(""PA"", 226.853)]",[]
5,NJ,2020-04-05,226.853,0.0,484.768,5067.14,6239.5,0.0,"[(""PA"", 226.853)]",[]
6,NJ,2020-04-06,226.853,0.0,425.269,5265.55,6239.5,0.0,"[(""PA"", 226.853)]",[]
7,NJ,2020-04-07,226.853,0.0,466.18,5504.88,6239.5,0.0,"[(""PA"", 226.853)]",[]
8,NJ,2020-04-08,108.72,0.0,734.62,6130.78,6239.5,0.0,"[(""PA"", 108.72)]",[]
9,NJ,2020-04-09,0.0,39.0066,44.7757,6214.56,6239.5,0.0,[],"[(""MA"", 39.0066)]"


In [15]:
println("Connectivity:")
Dict(ne_states[i] => ne_states[row] for (i,row) in enumerate(eachrow(adj)))

Connectivity:


Dict{String,Array{String,1}} with 11 entries:
  "RI" => ["CT", "DE", "MA", "ME", "NH", "NJ", "NY", "VT"]
  "NJ" => ["CT", "DE", "MA", "MD", "NY", "PA", "RI"]
  "DE" => ["CT", "MD", "NJ", "NY", "PA", "RI"]
  "MD" => ["DE", "NJ", "NY", "PA"]
  "NH" => ["CT", "MA", "ME", "NY", "RI"]
  "NY" => ["CT", "DE", "MA", "MD", "NH", "NJ", "PA", "RI", "VT"]
  "ME" => ["CT", "MA", "NH", "RI", "VT"]
  "CT" => ["DE", "MA", "ME", "NH", "NJ", "NY", "RI", "VT"]
  "MA" => ["CT", "ME", "NH", "NJ", "NY", "RI", "VT"]
  "PA" => ["DE", "MD", "NJ", "NY"]
  "VT" => ["CT", "MA", "ME", "NY", "RI"]

In [16]:
println("Sent to:")
Dict(ne_states[i] => ne_states[row] for (i,row) in enumerate(eachrow(sum(sent, dims=3)[:,:,1] .> 0)))

Sent to:


Dict{String,Array{String,1}} with 11 entries:
  "RI" => ["NY"]
  "NJ" => ["PA"]
  "DE" => String[]
  "MD" => String[]
  "NH" => String[]
  "NY" => ["MD", "PA"]
  "ME" => String[]
  "CT" => ["DE", "ME", "NY", "VT"]
  "MA" => ["NH", "NJ"]
  "PA" => String[]
  "VT" => String[]